# DATASET INFORMATION

*http://xiaoyongshen.me/webpage_portrait/index.html dataset su 
* 1597 tane image ve pixel level mask var(800x600) size
*  1397 tanesi trainingte, 100 tanesi valde, 100 tanesi test kullanildi.

In [0]:
def meaniou(prediction,target):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score
    
sum = 0
for i in range(len(predictions)):
    sum += meaniou(predictions[i], label[i])t
    
print(sum/len(predictions))

# MODEL INFORMATION

* Modelin yaptigi semantic segmentation deniliyor.
* Model architecture UNet ( Encoder ve Decoder parcalari var ve encoder ve decoder arasinda residual connectionlar var  featurelari almak icin encoder partindan.)
* Model encoder tarafinda MobilenetV2 denilen model var. Bu modeli kullanmamizin nedeni imagenetteki featurelari cikarma basarisi ve kucuk, efficient olmasi.
* Model decoder partini biz yazdik.MobileNetv2 ara layerlarindan feature mapleri aliyor( residual connection)
* Loss olarak iki lossun birlesimini kullandik. 
**Dice Loss: Intersection Over Union Oranini artirmaya calisiyor.
<br>
-------------------code-----------------------------
modely.py icinde
----------------------------------------------------
<br>
**Binary Cross Entropy Loss:Output ve Target Masklerinin pixel similarity artirmaya calisiyor(pixel wise loss)(her pixel icin)
*Metric : Mean of Intersection Over Union Orani : bizim model %96.1 (MIou)
<br>
-------------------code-----------------------------
def meaniou(prediction,target):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score
    
sum = 0
for i in range(len(predictions)):
    sum += meaniou(predictions[i], label[i])t
    
print(sum/len(predictions))

--------------------------------------------------------
    
<br>
*Nvidia 950M : ~15-20 FPS
*Skype Blur ozelligi buna benzer bir AI kullaniyor.

#PREPROCESSING

*Imagelerin verilen bound gore croplanmasi gerekiyordu. Bir resim 1200x900 olabiliyor ama tum masklar 800x600 boyutunda

In [0]:
def load_crop_rects(filepath):
    # Load file
    
    with open(filepath, 'r') as f:
        lines = f.read().strip().split('\n')

    rect_pairs = list()
    for line in lines:
        items = line.split()
        if len(items) != 5:
           
            continue
      
        rect_pairs.append((items[0], items[1:5]))

    return rect_pairs

def crop_img(img_name, src_dir, dst_dir, crop_rect, img_size):
    src_path = os.path.join(src_dir, img_name)
    if not os.path.exists(src_path):
        
        return

    dst_path = os.path.join(dst_dir, img_name)
    if os.path.exists(dst_path):
        
        return

    img = cv2.imread(src_path)
    x0, y0 = int(crop_rect[2]), int(crop_rect[0])
    x1, y1 = int(crop_rect[3]), int(crop_rect[1])
    img = img[y0:y1, x0:x1, :]
    img = cv2.resize(img, img_size)
    cv2.imwrite(dst_path, img)
    print(dst_path)

# Random Cropping
 
 * Model sadece square size ( m x m ) image kabul ettigi icin 512x512 random crop aldik her image ve masktan 5 tane elde ettik toplamda 1597 * 5 dataset oldu. 512x512 secmemizin nedeni 32 bolunmesi ( maxpooling icin) ve precision icin buyuk imageler icin.Normalde pretrained model ( MobilenetV2 224x224 aliyor)

In [0]:
import random
count =  1 
names = random.sample(range(int(1e10)), len(imgs) * 20)

os.system("mkdir augtrain_data")
os.system("mkdir augtrain_label")
for i in range(len(imgs)-200):
    for k in range(5):
        j = imgs_mask[i]
        rand1 = np.random.randint(j.shape[0]-513)
        rand2 = np.random.randint(j.shape[1]-513)
        a = cv2.cvtColor(imgs[i], cv2.COLOR_RGB2BGR)
        cv2.imwrite("./augtrain_data/{}.png".format(names[count]),a[rand1:rand1+512,rand2:rand2+512,:])
        cv2.imwrite("./augtrain_label/{}.png".format(names[count]),j[rand1:rand1+512,rand2:rand2+512])
        count +=1
        
os.system("mkdir augval_data")
os.system("mkdir augval_label")        
for i in range(len(imgs)-200, len(imgs) -100):
    for k in range(5):
        j = imgs_mask[i]
        rand1 = np.random.randint(j.shape[0]-513)
        rand2 = np.random.randint(j.shape[1]-513)
        a = cv2.cvtColor(imgs[i], cv2.COLOR_RGB2BGR)
        cv2.imwrite("./augval_data/{}.png".format(names[count]),a[rand1:rand1+512,rand2:rand2+512,:])
        cv2.imwrite("./augval_label/{}.png".format(names[count]),j[rand1:rand1+512,rand2:rand2+512])
        count +=1
        
os.system("mkdir augtest_data")
os.system("mkdir augtest_label")        
for i in range(len(imgs)-100, len(imgs)):
    for k in range(5):
        j = imgs_mask[i]
        rand1 = np.random.randint(j.shape[0]-513)
        rand2 = np.random.randint(j.shape[1]-513)
        a = cv2.cvtColor(imgs[i], cv2.COLOR_RGB2BGR)
        cv2.imwrite("./augtest_data/{}.png".format(names[count]),a[rand1:rand1+512,rand2:rand2+512,:])
        cv2.imwrite("./augtest_label/{}.png".format(names[count]),j[rand1:rand1+512,rand2:rand2+512])
        count +=1
        

# FURTHER DATA AUGMENTATION 
 
 *Image pixelleri rescale edildi img /= 255.0 --> pixel degerleri [0,1], amac weightlerin stabilitisi( masklerede uygulandi)
<br>
** Brightness Augmentation : Color Space HSV cevirildi, brightness parti oldugu icin bu color space kullanildi. Random sayilarla carpilip, datalarin brightness degerleri degistirildi. Tekrardan HSV den RGB color space donuldu.Amac farkli isik conditionlarina alistirmak
<br>
** Quality Augmentation : Opencv kullanilarak resim okunup memory bufferina random olarak dusuk quality olarak kaydedildi. Sonra bufferdan okundu. Amac farkli qualitydeki resimlere alistirmak. Ayni size quality dusurdum.



In [0]:
img = cv2.cvtColor(cv2.imread("test3.jpg"),cv2.COLOR_BGR2RGB)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
def brightness_adjustment(img):
    #print(img.shape)
    if(img.shape[2] == 3):
      # turn the image into the HSV space
      hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
      # creates a random bright
    
      ratio = .5 + np.random.uniform(-0.499,+0.499)
      # convert to int32, so you don't get uint8 overflow
      # multiply the HSV Value channel by the ratio
      # clips the result between 0 and 255
      # convert again to uint8
      hsv[:,:,2] =  np.clip(hsv[:,:,2].astype(np.int32) * ratio, 0, 255).astype(np.uint8)
      # return the image int the BGR color space
      return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB).astype(np.uint8)


def img_compression(img):
    if img.shape[2] == 3:
    
      encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50 + np.random.uniform(-40,20)]
      result, encimg = cv2.imencode('.jpg', img, encode_param)
      img = cv2.imdecode(encimg, 1)
      
      return brightness_adjustment( img).astype(np.float64)
    return img

plt.imshow(img_compression(img).astype(np.uint8))

# TRAINING

In [0]:
from modely import *
from keras.models import load_model
import random
size = 512
model = unet(input_size=(size,size,3) )#load_model("human_weights-004-0.1119.hdf5", custom_objects={"awesomeq_loss":awesomeq_loss})#unet(None,input_size = (size,size,3))
import os
import cv2
fname = "human_weights-{epoch:03d}-{val_loss:.4f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(fname, monitor="val_loss", save_best_only=True, verbose=1)
callbacks = [checkpoint]
from keras.preprocessing.image import ImageDataGenerator

batch = 16


def brightness_adjustment(img):
    #print(img.shape)
    if(img.shape[2] == 3):
      # turn the image into the HSV space
      hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
      # creates a random bright
      ratio = .5 + np.random.uniform(-0.499,0.799)
      # convert to int32, so you don't get uint8 overflow
      # multiply the HSV Value channel by the ratio
      # clips the result between 0 and 255
      # convert again to uint8
      hsv[:,:,2] =  np.clip(hsv[:,:,2].astype(np.int32) * ratio, 0, 255).astype(np.uint8)
      # return the image int the BGR color space
      return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB).astype(np.uint8)
    
    return img
  
def img_compression(img):
    if img.shape[2] == 3:
    
      encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50 + np.random.uniform(-40,20)]
      result, encimg = cv2.imencode('.jpg', img, encode_param)
      img = cv2.imdecode(encimg, 1)
      
      return brightness_adjustment( img).astype(np.float64)
    return img

data_gen_args = dict(preprocessing_function= img_compression,rescale = 1/255.0)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
seed= 1

image_generator = image_datagen.flow_from_directory(
    'train',
    class_mode=None,
    target_size = (size,size),
    classes = ['data'],
    color_mode ='rgb',
    batch_size = batch,
    shuffle=True,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'train',
    class_mode=None,
    classes = ['label_1'],
    target_size = (size,size),
    color_mode = 'grayscale',
    batch_size = batch,
    
    seed=seed)

train_generator = zip(image_generator, mask_generator)

vimage_generator = image_datagen.flow_from_directory(
    'val',
    class_mode=None,
    classes = ['data'],
    target_size = (size,size),
    color_mode ='rgb',
    batch_size = batch,
    shuffle = True,
    seed=seed)

vmask_generator = mask_datagen.flow_from_directory(
    'val',
    class_mode=None,
    classes = ['label_1'],
    target_size = (size,size),
    color_mode = 'grayscale',
    batch_size = batch, 
    seed=seed)
val_generator = zip(vimage_generator, vmask_generator)

from keras.optimizers import SGD,Adam, RMSprop

optimizer =  RMSprop(5e-5)
model.compile(optimizer = Adam(lr = 5e-5), loss = awesomeq_loss, metrics = ['accuracy'])


H = model.fit_generator(
        train_generator,
	      callbacks = callbacks,
        steps_per_epoch=6985//batch,
        epochs=20,
        verbose = 1,
        validation_data=val_generator,
        validation_steps=500//batch)

import numpy as np 
import matplotlib.pyplot as plt 

plt.style.use("ggplot")
fig = plt.figure()


plt.plot(np.arange(0, 20), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 20), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 20), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 20), H.history["val_acc"], label="val_acc")
title = "0.01"
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch#")

plt.ylabel("Loss/Accuracy")
plt.legend()
fig.savefig('image.png', dpi=fig.dpi)
plt.show()